##### 问题:
我们想定义一个函数或者方法，其中有一个或多个参数是可选的并且带有默认值。

##### 解决方案:
表面上看定义一个带有可选参数的函数是非常简单的——只需要在定义中为参数赋值，并确保默认参数出现在最后即可。示例如下：

In [73]:
def spam(a, b=42):
    print(a, b)

spam(1)
spam(1, 2)

1 42
1 2


如果默认值是可变容器的话，比如说列表、集合或者字典，那么应该把None作为默认值，代码应该像这样编写：

In [74]:
def spam(a, b=None):
    if b is None:
        b = []

如果不打算提供一个默认值，只是想编写代码来检测可选参数是否被赋予了某个特定的值，那么可以采用下面的惯用手法：

In [75]:
_no_value = object() 
def spam(a, b=_no_value): 
    if b is _no_value:
        print('No b value supplied') 

spam(1)


No b value supplied


In [76]:
spam(1, 2) # 无输出

In [77]:
spam(1, None) # 无输出

请仔细区分不传递任何值和传递None之间的区别。

定义带有默认参数的函数看似很容易，但其实并不像看到的那么简单。首先，对默认参数的赋值只会在函数定义的时候绑定一次。可用下面这个例子做下试验：


In [78]:
x = 42
def spam(a, b=x):
    print(a, b)

spam(1)

1 42


In [79]:
x = 23 #没有影响函数的初始值
spam(1)

1 42


注意到修改变量x的值（x被作为函数参数的默认值）并没有对函数产生任何效果。这是因为默认值已经在函数定义的时候就确定好了。

其次，给默认参数赋值的应该总是不可变的对象，比如None、True、False、数字或者字符串。特别要注意的是，绝对不要编写这样的代码：

def spam(a, b=[]):        # NO!


如果这么做了就会陷入到各种麻烦之中。如果默认值在函数体之外被修改了，那么这种修改将在之后的函数调用中对参数的默认值产生持续的影响。示例如下：

In [80]:
def spam(a, b=[]):
    print(b)
    return b
x = spam(1)
print(x)

[]
[]


In [81]:
x.append(99) 
x.append('Yow!') 
print(x)



[99, 'Yow!']


In [82]:
spam(1)

[99, 'Yow!']


[99, 'Yow!']

这很可能不是所期望的结果。要避免出现这种问题，最好按照解决方案中的做法，使用None作为默认值并在函数体中增加一个对默认值的检查

当检测默认参数是否为None时，本节示例的关键之处在于对is操作符的运用。有时候人们会犯这样的错误：

In [83]:
def spam(a, b=None): 
    if not b:     # NO! Use 'b is None' instead 
        b = []

这里出现的问题在于尽管None会被判定为False，可是还有许多其他的对象（比如长度为0的字符串、列表、元组、字典等）也存在这种行为。因此，上面示例给出的条件检测会将某些特定的输入也判定为False，从而错误地忽略掉这些输入值。示例如下：

<div>
<div>>>> spam(1)          # OK
<div>>>> x = [] 
<div>>>> spam(1, x)       # Silent error. x value overwritten by default
<div>>>> spam(1, 0)       # Silent error. 0 ignored
<div>>>> spam(1, '')      # Silent error. '' ignored


本节最后讨论的内容更加巧妙——在函数中检测是否对可选参数提供了某个特定值（可以是任意值）。这里最为棘手的地方在于我们不能用None、0或者False当做默认值来检测用户是否提供了参数（因为所有这些值都是完全合法的参数，用户极有可能将它们当做参数）。因此，需要用其他的办法来检测。

要解决这个问题，可以利用object()创建一个独特的私有实例，就像解决方案中给出的那样（即，变量_no_value）。在函数中，可以用这个特殊值来同用户提供的参数做相等性检测，以此判断用户是否提供了参数。这里主要考虑到对于用户来说，把_no_value实例作为输入参数几乎是不可能的。因此，如果要判断用户是否提供了某个参数，_no_value就成了一个可以用来安全比较的值。

这里用到的object()可能看起来很不常见。object作为Python中几乎所有对象的基类而存在。可以创建object的实例，但是它们没有任何值得注意的方法，也没有任何实例数据，因此一般来说我们对它是毫无兴趣的（因为底层缺少__dict__字典，我们甚至没法为它设置任何属性）。唯一可做的就是检测相等性，这也使得它们可作为特殊值来使用，就像我们给出的解决方案中那样。
